### Import packages and load data:

In [ ]:
import pandas as pd
import re
from itertools import accumulate
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

dfs = list()
for yr in range(2022, 2027):
    data = pd.read_csv(f"NCMAS {yr} outcomes.csv")
    data['year'] = yr
    dfs.append(data)

raw_data = pd.concat(dfs, ignore_index=True)
totall = "Total Allocation kSU"

alloc_by_year = raw_data.groupby('year').agg(
    proj_count=(totall, "count"),
    alloc_sum_kSU=(totall, "sum"),
    alloc_mean_kSU=(totall, "mean"),
    alloc_median_kSU=(totall, "median"),
    alloc_sd_kSU=(totall, "std"),
    alloc_min_kSU=(totall, "min"),
    alloc_max_kSU=(totall, "max")
)

Go8 = ['Australian National University',
       'University of Melbourne',
       'University of Sydney',
       'University of NSW',
       'University of Queensland',
       'Monash University',
       'University of Western Australia',
       'University of Adelaide']

alloc_Go8_by_year = raw_data.loc[raw_data['Institution'].isin(Go8)].groupby('year').agg(
    proj_count=(totall, "count"),
    alloc_sum_kSU=(totall, "sum"),
    alloc_mean_kSU=(totall, "mean"),
    alloc_median_kSU=(totall, "median"),
    alloc_sd_kSU=(totall, "std"),
    alloc_min_kSU=(totall, "min"),
    alloc_max_kSU=(totall, "max")
)

def for_seo_list(text, return_seo=False):
    codes = re.findall(r'\d{6}', str(text))
    percs = [float(i[:-1])/100 for i in re.findall(r'\d{1,3}%', str(text))]
    acc = list(accumulate(percs))
    try: 
        loc = acc.index(1.)+1
        forcodes = codes[:loc]
        forpercs = percs[:loc]
        seocodes = codes[loc:]
        seopercs = percs[loc:]
    except ValueError:
        forcodes = codes
        forpercs = percs
        seocodes = seopercs = []
    if return_seo:
        return forcodes, forpercs, seocodes, seopercs
    else:
        return list(forcodes), list(forpercs)

raw_data['FoR-codes'], raw_data['FoR-%'] = zip(*raw_data['FoR-08'].map(for_seo_list))
for_data = raw_data.explode(list(('FoR-codes', 'FoR-%')))
for_data['FoR-X'] = for_data['FoR-codes'].map(lambda t:str(t)[:1])
for_data['FoR-XX'] = for_data['FoR-codes'].map(lambda t:str(t)[:2])
for_data['FoR-kSU'] = for_data[totall]*for_data['FoR-%']

### Resulting DataFrames:

- `raw_data`: All `outcomes.csv` entries with an additional "year" column

- `alloc_by_year`: Allocation statistics aggregated by year.

- `alloc_Go8_by_year`: Allocation statistics aggregated by year for only Go8 institutions.

- `for_data`: As per `raw_data` but each FoR code is listed with an "FoR-kSU" column breaking up the Total Allocation kSU by FoR%.

For example, a `raw_data` allocation of 1,000 kSU with FoR codes 60% 123456 and 40% 654321 will have two `for_data` rows:

```
... Lead Investigator Total Allocation kSU FoR-codes FoR-kSU
... Ash Fairenough    1000                 123456    600
... Ash Fairenough    1000                 654321    400
```

### Example viz: Histogram of Allocation Frequency by Size for different years, and Allocation Statistics by Year and Institution (Go8 / not)

In [ ]:
fig, ax = plt.subplots()

for year in range(2023, 2027):
    data = raw_data.loc[raw_data['year'] == year, 'Total Allocation kSU']
    hist, bin_edges = np.histogram(data, bins=32, range=(-5000, 55000))
    bincent = 0.5*(bin_edges[1:]+bin_edges[:-1])
    ax.plot(bincent/1000, hist, label=year)
ax.legend()
ax.set_xlabel('Alloc size (,000 kSU)')
ax.set_ylabel('Alloc frequency')
ax.set_title('NCMAS allocation frequency by size, 2023-2026')
plt.show()

In [ ]:
fig, axs = plt.subplots(3,2, figsize=(6,10))
ax = axs.flatten()
for i, col in enumerate(['proj_count', 'alloc_sum_kSU', 'alloc_mean_kSU', 'alloc_median_kSU', 'alloc_min_kSU', 'alloc_max_kSU']):
    mult = (1 if i==0 else 0.001)
    ax[i].plot(alloc_by_year.index, mult*alloc_by_year[col], 'o-', label="All inst.")
    ax[i].plot(alloc_Go8_by_year.index, mult*alloc_Go8_by_year[col], 'o-', label="Go8")
    ax[i].set_ylim(0, [170, 950, 11, 10, 5, 60][i])
    ax[i].set_ylabel(['No. projects', 'Total alloc. (,000 kSU)', 'Mean alloc. (,000 kSU)', 'Median alloc. (,000 kSU)', 'Min alloc. (,000 kSU)', 'Max alloc. (,000 kSU)'][i])
    ax[i].set_xlabel('Year')
    ax[i].legend()
fig.suptitle('NCMAS Allocation Statistics by Year and Institutions')
fig.tight_layout()